1) For Cube puzzles, we know there are rules of __mi.mi.mi = -mi__ and __mi.mi.mi.mi.= empty__. Let's make it more general for all the three kinds of puzzles into the sign change rules with the form of __mi\*\*(n) = -mi\*\*(n)__:

- For Cube, __n=2__, or say __mi.mi = -mi.-mi__. By the pair cancellation rule (https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles), we can get:
    - mi.mi.mi = -mi.-mi.mi = -mi
    - mi.mi.mi.mi = -mi.-mi.mi.mi = empty
    - Meanwhile, there are commutative rule for cube. i.e. mi.m?.mi.m?.mi = unordered(-mi.m?.m?)
    
    
- For Wreath, n is based on the size of the wreath.
    - when wreath size is an even number, __n = size/2__. We could imagine the circle has the symmetric feature. i.e. wreath_6/6 has n=3
    - when wreath size is an odd number, __n = size__. i.e. wreath_7/7 has n=7
    - Based on the tests, the wreath is non-commutative, so I only implement replacement without location change, similar to https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles
    
    
- For Globe: 
    - fi is non-commutative with fi=-fi (https://www.kaggle.com/code/cl12102783/cancel-pairs-for-all-puzzles) or n=1
    - ri is commutative and n equals the final number of the type. i.e. globe_1/8 has n=8
    
2) Furthermore, to apply the sign change rule, we could think when the count of the mi is larger than n, then there will be downsizing. The downsizing rule is __n_left = n-(count - n)__:
   - when n_left>0, there will __-mi\*\*(n_left)__. i.e. for cube, mi.mi.mi has n=2, so 2-(3-2)=1, to be -mi
   - when n_left<0, there will __mi\*\*(n_left)__. i.e. for globe_1/8, ri\*\*(18) has n=8, so 8-(18-8)=-2, to be ri\*\*(2)
   - when n_left=0, all mi will be cancelled in the same move group. i.e. for cube, mi.mi.mi.mi has n=2, so 2-(4-2)=0, to be cancelled


In [2]:
import pandas as pd
from collections import deque
import tqdm
import glob
import numpy as np
from collections import Counter

df_puzzles = pd.read_csv('puzzles.csv')
df_puzzle_info = pd.read_csv('puzzle_info.csv')

# Consider all high-score files
files = pd.read_csv('7ocakbest.csv')

In [3]:
def cancel_group(moves, group):
    def get_grp(elem):
        return elem[1] if elem.startswith('-') else elem[0]

    def drop_grp(elem, group):
        grp = group.split('_')[0]
        if grp == 'globe' and elem[-1].replace('-','')[0]=='f':
            return elem
            
        config = dict(
            cube = 2,
            globe = int(group.split('/')[-1]),
        )
        grp_size = config[grp]
        move = elem.copy()
        flag = True
        while flag:
            flag = False
            cnt = Counter(move)
            for k, v in cnt.items():
                if v > grp_size:
                    flag = True
                    sign_opp = '' if k.startswith('-') else '-'
                    num_add = grp_size-(v-grp_size)
                    for _ in range(abs(num_add)):
                        if num_add>0:
                            move.append(sign_opp+k.replace('-',''))
                        elif num_add<0:
                            move.append(k)
                    for _ in range(v):
                        move.remove(k)
                    break
        return move
    
    def solver(elem, grp):
        move = elem.copy()
        grp_name = grp.split('_')[0]
        if grp_name !='wreath':
            move = drop_grp(move, grp)
        return move
    
    moves = moves+'.'
    win = deque()
    result = deque()
    move = ''
    for i in moves:
        if i != '.':
            move += i
        else:
            if len(win)<1:
                win.append(move)
                move = ''
                continue
            grp_last, grp_new = get_grp(win[-1]), get_grp(move)
            if grp_last == grp_new:
                win.append(move)
            else:
                result.extend(solver(win, group))
                win = deque([move])
            move=''
     
    # Collect remainder
    if len(win)>0:
        result.extend(solver(win, group))
    return '.'.join(result)

def drop_grp_wreath(moves, group):
    kind = group.split('/')[-1]
    kind = int(int(kind)/2) if int(kind)%2==0 else int(kind)
    moves += '.'
    result = deque()
    win = deque()
    move = ''
    
    for i in moves:
        if i !='.':
            move+=i
        else:
            if not win:
                win.append(move)
                move = ''
                continue
            if move != win[-1]:
                if len(win) > kind:
                    print('I am here')
                    times = kind-(len(win)-kind)
                    if times>0:
                        case_opp = win[-1][1] if len(win[-1])>1 else '-'+win[-1]
                        result.extend([case_opp]*times)
                    elif times<0:
                        result.extend([win[-1]]*abs(times))
                else:
                    result.extend(win)
                win = deque([move]) 
            else:
                win.append(move)
            move = ''
    if win:
        result.extend(win)
    return '.'.join(result)

def multiple_try(elem, group):
    len_old = len(elem.split('.'))
    move_old = elem
    flag = True
    while flag:
        if group.split('_')[0] == 'wreath':
            move_new = drop_grp_wreath(move_old, group)
        else:
            move_new = cancel_group(move_old, group)
            
        len_new = len(move_new.split('.'))
        if len_new<len_old:
            move_old = move_new
            len_old = len_new
        else:
            flag=False
    return move_old

In [5]:
result = dict()

for row in files:
    
    id_, move = row.strip().split(',')
    id_ = int(id_)
    group = df_puzzles.iloc[id_].puzzle_type
    
    move = multiple_try(move, group)
    
    if id_ not in result:
        result[id_] = move
    else:
        if len(move.split('.')) < len(result[id_].split('.')):
            result[id_] = move

ValueError: not enough values to unpack (expected 2, got 1)

In [4]:
df_sub = pd.DataFrame()
df_sub['id'] = result.keys()
df_sub['moves'] = result.values()
df_sub.to_csv('submission.csv', index=False)

In [5]:
df_sub.moves.str.split('.').apply(lambda x: len(x)).sum()

827386